In [1]:
# // 1.  Import packages that we need:
import numpy as np
import pandas as pd
# // Web scraping: 
import requests
import string
from bs4 import BeautifulSoup
# // OS. Sometimes need this for finding working directory:
import os
# // datetime
import datetime
# ////////////////////////////////////////////////////////////////

In [2]:
url_base = "https://www.tidetimes.org.uk/plymouth-devonport-tide-times-{}"

# create empty list
days = []

# loop i times to get dates for next seven days, adds to list in string form to work with url
for i in range(7):
    days.append((datetime.datetime.now() + datetime.timedelta(days=i)).strftime('%Y%m%d'))

# // Set up a blank DataFrame to fill up:
df = pd.DataFrame()

for d in days:

    # // Return the index number of the thing we are working with:
    s = days.index(d)

    # // Build the URL for this iteration of the loop:
    URL = url_base.format(d)

    # // Request the html from the URL:
    html = requests.get(URL)

    # // Get the soup of this page
    soup = BeautifulSoup(html.content, 'html.parser')

    # // Now get what we want from the page: 
    soup_hilo = soup.find_all("td", class_="tal")
    soup_time = soup.find_all("td", class_="tac")
    soup_height = soup.find_all("td", class_="tar")

    # // Take only text
    soup_hilo=[i.text for i in soup_hilo]
    soup_time=[i.text for i in soup_time]
    [i.text for i in soup_height]


    # ---------------- Clean hilo --------------------- #
    # // Clean hilo - Delete first value in list 
    del soup_hilo[:1]


    # ---------------- Clean time --------------------- #
    # some days only have 3 tide changes
    tidechanges = len(soup_hilo)
    # // Create new empty list, and combine dates with HHMM times
    timestamp = []
    # If else:
    if tidechanges==4:
        soup_time.remove(soup_time[0])
        soup_time.remove(soup_time[4]) #removing first element means 5th element becomes 4th
        for i in range(4):
            date = str(days[days.index(d)]) + ' ' + soup_time[i]
            timestamp.append(date[:4] + "-" + date[4:6] + "-" + date[6:])
        # Code to run if true
    else:
        soup_time.remove(soup_time[0])
        soup_time.remove(soup_time[3]) #removing first element means 4th element becomes 3th
        for i in range(3):
            date = str(days[days.index(d)]) + ' ' + soup_time[i]
            timestamp.append(date[:4] + "-" + date[4:6] + "-" + date[6:])
        # Code to run if condition is FALSE


    # ---------------- Clean heights --------------------- #
    # // Remove first entry in list
    soup_height.remove(soup_height[0])
    # // create new list without 'm' on each entry
    heights=[int(i.text.replace('m','').replace('.','')) for i in soup_height]

    # // Group together:
    results = [timestamp, heights, soup_hilo]

    # // Convert array into dataframe:
    # // This will change with each iteration of the loop:
    # // [This is why I use subscript d, as a reminder]
    df_d = pd.DataFrame(results)

    # // Name this column of data:
    # // I want this column to update, named to be the name of the ticker i.e. 1, 2, 3.
    #df_d.columns = ['Data1', 'Data2', 'Data3', 'Data4']

    # // Add this to# the master dataframe:
    #df[s] = df_d['Data1']
    #df[s] = df_d['Data2']
    #df[s] = df_d['Data3']
    #df[s] = df_d['Data4']

    df = pd.concat([df, df_d], axis=1)


In [3]:
# transpose this
df = df.T
df.reset_index(inplace=True, drop=True)
df.head()

,0,1,2
0,2022-01-11 05:52,215,Low
1,2022-01-11 12:09,454,High
2,2022-01-11 18:28,212,Low
3,2022-01-12 00:48,437,High
4,2022-01-12 06:58,227,Low


In [4]:
# // Now add the names as the names of the columns:   
df.columns = ['time', 'height', 'hilo']
df.head()

,time,height,hilo
0,2022-01-11 05:52,215,Low
1,2022-01-11 12:09,454,High
2,2022-01-11 18:28,212,Low
3,2022-01-12 00:48,437,High
4,2022-01-12 06:58,227,Low


In [5]:
# change Dtype for 'time' from object to datetime
df['time'] = pd.to_datetime(df['time'])
# convert cm back to m for height
df['height'] = pd.to_numeric(df['height']/100)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   time    27 non-null     datetime64[ns]
 1   height  27 non-null     float64       
 2   hilo    27 non-null     object        
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 776.0+ bytes


,time,height,hilo
0,2022-01-11 05:52:00,2.15,Low
1,2022-01-11 12:09:00,4.54,High
2,2022-01-11 18:28:00,2.12,Low
3,2022-01-12 00:48:00,4.37,High
4,2022-01-12 06:58:00,2.27,Low


In [6]:
# export as JSON
df.to_json('/Users/joshhellings/Documents/OneDrive - University of Bristol/Economics Year 3/Data Science/Github Mirror/Portfolio/data/Week 7 Scraping/7dayforecast.JSON', orient='records')